In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
### from sklearn.cluster import hierarchical
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering 
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-whitegrid')
pd.pandas.set_option('display.max_columns',None)

In [ ]:
df = pd.read_csv("C:\COVID\covid-data_cis7031.csv")
df

### 1. Data Processing

#### 1.1.	Download the dataset from the link above and create a dataframe that contains Europe data only.

In [ ]:
df_Europe = df[df.continent == 'Europe']
df_Europe.reset_index(drop = True, inplace = True)
df_Europe

In [ ]:
df1 = df_Europe.copy()
df1.dropna(how='any')

#### Drop all the rows and columns where all the null values are present

In [ ]:
df1.dropna(how='all')

#### Dropping all the rows and columns if all NA values are present in a row and column

In [ ]:
df1 = df_Europe.copy()
pd.DataFrame({'MissingValues':df1.isnull().sum().values},index=df1.columns)

#### Checking for null values and sum up all the null values

In [ ]:
df1.isnull()

#### Checking for null values with boolean values.  True indicates null values and False indicates a no null values

In [ ]:
np.random.seed(0)
df2 = pd.DataFrame(np.random.randn(14577, 49), columns=list(df1.columns))
df2.iloc[::49] += np.random.randn() * 2  
Q1 = df1.quantile(0.25)
Q3 = df1.quantile(0.75)
IQR = Q3 - Q1
print((df2 < (Q1 - 1.5 * IQR)) | (df2 > (Q3 + 1.5 * IQR)))

In [ ]:
df0 = df1.fillna(0)
df0

#### Fill up all the null values with zeros

In [ ]:
df1.fillna(df1.mean(axis=0))

#### Calculating the mean where all the null values exist

In [ ]:
df2 = df0.copy()
df2.drop(df2.columns[df2.columns.str.contains('test|handwash|smoke|continent')], axis=1, inplace=True)
df2

#### Dropping all the columns related to the key words test, handwash, smoke and continent

### 2. Data Analysis

#### 2.1.	Which country has reported the highest and lowest covid cases over the period?

In [ ]:
dft = df0.copy()
dft_tot_cases_max = dft.groupby([dft['continent'],dft['location']]).agg({'total_cases':'max'}).reset_index()
dft_tot_cases_max

In [ ]:
dft_tot_cases_max['total_cases'] = dft_tot_cases_max['total_cases'].astype('Int64')
dft_tot_cases_max

In [ ]:
dft_tot_cases_max.plot.bar("location", "total_cases", rot = 90, fontsize = 13, color = 'g', figsize = (16,10))
plt.xlabel("Location", fontweight = 'bold', fontsize = 14)
plt.ylabel('Total number of cases',fontweight='bold',fontsize=14)
plt.title("LOCATION WISE TOTAL NUMBER OF COVID CASES IN EUROPE", fontweight = 'bold', fontsize = 14)
plt.legend(loc = "best", fontsize = '12')
plt.show()

#### The above bar graph reveals that France has reported the highest number of COVID cases and Vatican has reported the lowest number of cases over the period.  

#### 2.2.	Which country has the  has the highest and lowest deaths per million residents? 

In [ ]:
df3 = df0.copy()
df_tdpm = df3.groupby([df3['continent'],df3['location']]).agg({'total_deaths_per_million':'max'}).reset_index()
df_tdpm

In [ ]:
df_tdpm.plot.bar("location", "total_deaths_per_million", rot = 90, fontsize = 13, color = 'm', figsize = (16,10))
plt.xlabel("Location", fontweight = 'bold', fontsize = 14)
plt.ylabel('Total number of deaths per million',fontweight='bold',fontsize=14)
plt.title("LOCATION WISE TOTAL NUMBER OF DEATHS PER MILLION IN EUROPE", fontweight = 'bold', fontsize = 14)
plt.legend(loc = "best", fontsize = '12')
plt.show()

#### The above bar graph reveals that San Marino has reported the highest number of deaths per million and Faeroe Inslands has reported the lowest number of deaths per million

#### 2.3.	Which date was the highest number of cases reported and by which country? 

In [ ]:
df4 = df0.copy()

In [ ]:
df5 = df4.loc[df4.groupby('location')['new_cases'].idxmax()]
df5.sort_values(by='new_cases', ascending = False)

In [ ]:
df5.plot.bar("location", "new_cases", rot = 90, fontsize = 13, color = 'b', figsize = (16,10))
plt.xlabel("Location", fontweight = 'bold', fontsize = 14)
plt.ylabel('Highest number of cases',fontweight='bold',fontsize=14)
plt.title("COUNTRY WISE HIGHEST NUMBER OF CASES IN EUROPE", fontweight = 'bold', fontsize = 14)
plt.legend(loc = "best", fontsize = '12')
plt.show()

#### From the above obtained dataframe df5 and bar graph, it is revealed that on 08-11-2020 maximum number of COVID cases has been reported in France.  

#### 2.4.	Which date did UK report lowest number of cases?

In [ ]:
df_UK = df0[df0.location == 'United Kingdom']
df_UK.reset_index(drop = True, inplace = True)
df_UK

#### The above obtained dataframe df_UK shows that on 31-12-2019 United Kindom has reported the lowest number of COVID cases with '0' figure as United Kingdom is in its initial phase of the COVID pandemic.

### 3. Visual Analysis

#### Create a dynamic scatter/bubble plot showing the total cases per country over the period using Plotly express. Write your interpretation of the findings (~100 words).

In [ ]:
dft = df0.copy()
df_tcpc = dft.groupby([dft['continent'],dft['iso_code'],dft['location'],dft['date']]).agg({'total_cases':'max'}).reset_index()
df_tcpc
##df_tcpc = dft.groupby([dft['continent'],dft['iso_code'],dft['location']]).agg({'total_cases':'max'}).reset_index()
##df_tcpc['total_cases'] = dft_tot_cases_max['total_cases'].astype('Int64')
##df_tcpc

In [ ]:
df_tcpc.sort_values('date', ascending=True, ignore_index=True, inplace=True)

In [ ]:
fig = px.choropleth(df_tcpc, locations="iso_code",
                    color="total_cases", 
                    hover_name="location", animation_frame = 'date', hover_data=['location','total_cases'],scope = 'europe',
                    color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text = 'TOTAL NUMBER OF COVID CASES IN EUROPE', title_x=0.5, width=1000, height=700, 
                  margin={"r":0,"t":25,"l":0,"b":0})
fig.show()

#### The above Choropleth map shows the total number of COVID cases in Europe. France has reported the highest number of COVID cases and Vatican has reported the lowest number of COVID cases. As the cursor is hovered across each country, the details of country code and total number of cases are shown.

In [ ]:
df_world = df.copy()
df_world.fillna(0)
df_world = df_world.groupby([df_world['continent'],df_world['location'],df_world['date']]).agg({'total_cases':'max'}).reset_index()
df_world

In [ ]:
df_world.sort_values('date', ascending=True, ignore_index=True, inplace=True)

In [ ]:
fig = px.choropleth(df_world, locations="location", locationmode = 'country names', color="total_cases", 
                    hover_name="location", hover_data=['location','total_cases'], animation_frame = 'date',
                    color_continuous_scale=px.colors.sequential.Rainbow)
fig.update_layout(title_text = 'WORLD WIDE TOTAL NUMBER OF COVID CASES PER COUNTRY',
                  title_x=0.5, width=1000, height=600, margin={"r":0,"t":25,"l":0,"b":0})
fig.show()

#### The above Choropleth map shows the total number of COVID cases world wide. As the cursor is hovered across each country, the total number of COVID cases reported on a particular date with respect to that country is displayed.  There were no or less COVID cases during the intial phase i.e., in the month of December, 2019.  The total number of COVID cases has risen drastically with respect to time and is represented in the above choropleth map.

In [ ]:
dfp = df0.copy()
dfp = dfp.filter(['location','date','total_cases','total_deaths','population'], axis = 1)
dfp

In [ ]:
dfp.loc[:, dfp.dtypes == 'float64'] = dfp.loc[:, dfp.dtypes == 'float64'].astype('int64')

In [ ]:
dfp.sort_values('date', ascending=True, ignore_index=True, inplace=True)

In [ ]:
fig = px.scatter(dfp, x="total_cases", y="total_deaths", 
           animation_frame='date', animation_group="location",
           size="population", color="location", hover_name="location", log_x = True, log_y = False, size_max = 40, 
           range_y=[0,55000000])
fig.show()

#### The above bubble plot shows the total number of cases on x-axis and total deaths on y-axis.  The total time period is animated by taking the date values in the animation frame.  As the animation is played the plot shows the total cases and total deaths in bubbles format from the initial data to the final data taken.  As the cursur is hovered across the bubbles the data corresponding to the total cases and total deaths is shown.   

### 4. PCA/Correlation

In [ ]:
dfc = df0.copy()
dfc

In [ ]:
dfc.columns

In [ ]:
df_corr = dfc.groupby([dfc['location']]).agg({'total_cases':'max','new_cases':'max','total_deaths':'max'}).reset_index()
df_corr

In [ ]:
df_cor = df_corr.copy()
df_cor.set_index('location', inplace = True)
df_cor

In [ ]:
dfc_st = StandardScaler().fit_transform(df_cor)
dfc_st

In [ ]:
pca_c = PCA(n_components = 2)

In [ ]:
pca_c.fit(dfc_st)

In [ ]:
prin_com = pca_c.transform(dfc_st)
prin_com

In [ ]:
prin_df = pd.DataFrame(data = prin_com, columns = ['PC1', 'PC2'])
prin_df

In [ ]:
final_prin_df = pd.concat([prin_df, df_corr[['location']]], axis = 1)
final_prin_df

In [ ]:
plt.figure(figsize=(24,18))
sns.scatterplot(x='PC1', y='PC2', hue=final_prin_df.location.tolist(), data = final_prin_df, s=150)
plt.xlabel('PC1', fontsize = 20, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 20, fontweight = 'bold')
plt.xticks(fontsize=16,fontweight='bold')
plt.yticks(fontsize=16,fontweight='bold')
plt.title('2 Component PCA', fontsize = 24, fontweight = 'bold')
plt.legend(title = 'location', loc = 'best',fontsize='large', title_fontsize='20')
plt.show()

In [ ]:
plt.figure(figsize=(60,60))
sns.heatmap(prin_df,annot=True,cbar= True,square = True, cmap='RdYlGn', linewidths=0.5, annot_kws={'size':17})
plt.ylim([10,0])
plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(np.arange(51),['Albania','Andorra','Austria','Belarus','Belgium','Bosnia and Herzegovina','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Faeroe Islands','Finland',
                         'France','Germany','Gibraltar','Greece','Guernsey','Hungary','Iceland','Ireland','Isle of Man','Italy','Jersey','Kosovo','Latvia','Liechtenstien','Lithuania','Luxembourg',
                         'Macedonia','Malta','Moldova','Monaco','Montenegro','Netherlands','Norway','Poland','Portugal','Romania','Russia','San Marino','Serbia','Slovakia','Slovenia','Spain','Sweden',
                         'Switzerland','Ukraine','United Kingdom','Vativan'], fontsize=12,fontweight='bold', rotation = 45)
plt.title('2 COMPONENT PCA', fontsize = 16, fontweight = 'bold')
plt.show()

In [ ]:
pca_c.explained_variance_ratio_

#### The first two principal components comprises 96.1% of the information.  The first principal component comprises of the 71.2% variance and the second principal component comprises of 24.9% variance.  The remaining variance contains the rest of the dataset.

#### 4.2.	Is there a correlation between the number of deaths and age? Support your answer with discussion and plots.

In [ ]:
dfc_age_deaths = df0.copy()

In [ ]:
dfc_age_deaths = dfc_age_deaths.filter(['location','total_deaths','median_age','aged_65_older','aged_70_older'], axis = 1)
dfc_age_deaths

In [ ]:
dfc_age_deaths = dfc_age_deaths.corr()
dfc_age_deaths

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(dfc_age_deaths, annot=True, cbar= False, cmap = 'BrBG', linewidths=0.5, linecolor = 'w', annot_kws={'size':15})
#plt.ylim([10,10])
plt.xticks(fontsize=15,fontweight='bold', ha = 'center', rotation = 90)
plt.yticks(fontsize=15,fontweight='bold', ha = 'right', rotation = 0)
plt.title('CORRELATION BETWEEN THE NUMBER OF DEATHS AND AGE', fontsize = 16, fontweight = 'bold')
plt.show()

#### The heatmap obtained above shows that there is no correlatiion between the number of deaths and median age.  However, the number of deaths are more for the population age_65_older and aged_75_older.

### 5. Clustering

#### 5.1.	Using the hospital beds and GDP per capita ,undertake a K means clustering analysis (K=2 & 3) and identify countries cluster together.  Write your own interpretation (~100 words).

In [ ]:
dfk = dfc.groupby([dfc['location']]).agg({'hospital_beds_per_thousand':'max','gdp_per_capita':'max'}).reset_index()
dfk

In [ ]:
dfk_val = dfk.iloc[:, [1,2]].values

In [ ]:
opt_clusters = []
for i in range(1,11):
    kmeans=KMeans(n_clusters=i, init='k-means++', max_iter= 300, n_init= 10, random_state= 0)
    kmeans.fit(dfk_val)
    opt_clusters.append(kmeans.inertia_)
plt.figure(figsize=(13,7))
plt.plot(range(1, 11),opt_clusters)
plt.title('Optimum Number of Clusters - Elbow Method', fontsize = 18, fontweight = 'bold')
plt.xlabel('Number of clusters - K', fontsize = 14, fontweight = 'bold')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(fontsize=12,fontweight='bold')
plt.show()

In [ ]:
km_2=KMeans(n_clusters=2, init='k-means++', max_iter= 300, n_init= 10, tol = 1e-04, random_state= 0)
y_km2=km_2.fit_predict(dfk_val)
dfk['Km2']=y_km2

km_3=KMeans(n_clusters=3, init='k-means++', max_iter= 300, n_init= 10, tol = 1e-04, random_state= 0)
y_km3 = km_3.fit_predict(dfk_val)
dfk['Km3']=y_km3
dfk

#### Determining the optimum number of Clusters using the Elbow method.  The two elbows in the graph reveals the optimum number of clusters is 2 for the given dataframe.  

### k-means Clustering Analysis (k=2)

In [ ]:
plt.figure(figsize = (14,8))
plt.scatter(dfk_val[y_km2 == 0, 0], dfk_val[y_km2 == 0, 1], s = 100, c = 'red', edgecolor = 'black', label='Cluster 1')
plt.scatter(dfk_val[y_km2 == 1, 0], dfk_val[y_km2 == 1, 1], s = 100, c = 'green', edgecolor = 'black', label='Cluster 2')
plt.scatter(km_2.cluster_centers_[:, 0], km_2.cluster_centers_[:, 1], s = 100, c = 'blue', edgecolor = 'black', label='Centroids')
plt.title('Hospital Beds and GDP per capita (k-means - 2 Clustering Analysis)', fontsize = 18, fontweight = 'bold')
plt.xlabel('Hospital beds', fontsize = 14, fontweight = 'bold')
plt.ylabel('GDP per capita', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(fontsize=12,fontweight='bold')
plt.legend(loc='best', fontsize = 12)
plt.show()

### k-means Clustering Analysis (k=3)

In [ ]:
plt.figure(figsize = (14,8))
plt.scatter(dfk_val[y_km3 == 0, 0], dfk_val[y_km3 == 0, 1], s = 100, c = 'red', edgecolor = 'black', label='Cluster 1')
plt.scatter(dfk_val[y_km3 == 1, 0], dfk_val[y_km3 == 1, 1], s = 100, c = 'green', edgecolor = 'black', label='Cluster 2')
plt.scatter(dfk_val[y_km3 == 2, 0], dfk_val[y_km3 == 2, 1], s = 100, c = 'blue', edgecolor = 'black', label='Cluster 3')
plt.scatter(km_3.cluster_centers_[:, 0], km_3.cluster_centers_[:, 1], s = 100, c = 'orange', edgecolor = 'black', label='Centroids')
plt.title('Hospital beds and GDP per capita (k-means - 3 Clustering Analysis)', fontsize = 18, fontweight = 'bold')
plt.xlabel('Hospital beds', fontsize = 14, fontweight = 'bold')
plt.ylabel('GDP per capita', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(fontsize=12,fontweight='bold')
plt.legend(loc='best', fontsize = 12)
plt.show()

#### The above two scatter plots reveal that the given dataframe with the data hospital beds and GDP per capita can be divided into two and three clusters (K=2 and K=3) respectively.  The scatter plots clearly reveal that there is a clear seperation of data points into clusters.  The data for GDP per capita is divided into 3 clusters.  Cluster-1 with GDP per capita under 20K, Cluster-2 wih GDP per capita between 25K to 50K and Cluster-3 with GDP per capita above 55K.  The clustering of data into 2 clusters is not recommended because the countries with higher GDP are brackted into medium GDP countries and there is no clear demarkation into high, medium and low GDP countries.  The higher the GDP per capita the lower the hospital beds and vice versa.

### 5.2.	Using the same dataset (5.1) create a hierarchical cluster.  Compare the cluster with k means clusters (~100 words).

In [ ]:
dfh = dfk.drop(dfk.columns[[3,4]], axis=1)
dfh

In [ ]:
dfh.set_index('location', inplace = True)
dfh

In [ ]:
dfh_val = dfh.iloc[:, [0,1]].values

In [ ]:
plt.figure(figsize=(18,12))
dendgram = sch.dendrogram(sch.linkage(dfh_val, method  = "ward"), labels=dfh.index, leaf_rotation=90, leaf_font_size=14)
plt.title('Hierarchical Clustering Dendrogram - 2 Clusters', fontsize = 18, fontweight = 'bold')
plt.xlabel('Location', fontsize = 14, fontweight = 'bold')
plt.ylabel('Euclidean distance', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 13, fontweight = 'bold')
plt.yticks(fontsize = 11, fontweight = 'bold')
plt.axhline(y=100000, color='maroon', linestyle='--')
plt.show()

### Line cuts the dendrogram at 2 points.  Hence 2 Clusters

In [ ]:
plt.figure(figsize=(18,12))
dendgram = sch.dendrogram(sch.linkage(dfh_val, method  = "ward"), labels=dfh.index, leaf_rotation=90, leaf_font_size=14)
plt.title('Hierarchical Clustering Dendrogram - 3 Clusters', fontsize = 18, fontweight = 'bold')
plt.xlabel('Location', fontsize = 14, fontweight = 'bold')
plt.ylabel('Euclidean distance', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 13, fontweight = 'bold')
plt.yticks(fontsize = 11, fontweight = 'bold')
plt.axhline(y=60000, color='maroon', linestyle='--')
plt.show()

#### Line cuts the dendrogram at 3 points.  Hence 3 Clusters

#### Hierarchial Clustering solves the problem of K- Means clustering.  The data is divided into three clusters clearly.  The Dendogram in clustering - 2 divides the GDP per capita into 2 clusters showing the low and high values.  Clustering - 3 divides the dendogram into 3 clusters high, medium and low.  Dividing the dataset into 3 clusters into low, medium and high is the best as can be deduced from the dendograms.

## Hierarchical Clustering Visualization - 2 & 3 Clusters

In [ ]:
hc_2 = AgglomerativeClustering(n_clusters = 2, affinity = 'euclidean', linkage ='ward')
y_hc2 = hc_2.fit_predict(dfh_val)
dfh['hc2']=y_hc2

hc_3 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='ward')
y_hc3 = hc_3.fit_predict(dfh_val)
dfh['hc3']=y_hc3

In [ ]:
dfh.reset_index(inplace=True)
dfh

In [ ]:
plt.figure(figsize = (14,8))
plt.scatter(dfh_val[y_hc2 == 0, 0], dfh_val[y_hc2 == 0, 1], s = 100, c = 'red', edgecolor = 'black', label='Cluster 1')
plt.scatter(dfh_val[y_hc2 == 1, 0], dfh_val[y_hc2 == 1, 1], s = 100, c = 'green', edgecolor = 'black', label='Cluster 2')
plt.title('Hospital beds and GDP per capita (Hierarchical - 2 Clustering Analysis)', fontsize = 18, fontweight = 'bold')
plt.xlabel('Hospital beds', fontsize = 14, fontweight = 'bold')
plt.ylabel('GDP per capita', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(fontsize=12,fontweight='bold')
plt.legend(loc='best', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize = (14,8))
plt.scatter(dfh_val[y_hc3 == 0, 0], dfh_val[y_hc3 == 0, 1], s = 100, c = 'red', edgecolor = 'black', label='Cluster 1')
plt.scatter(dfh_val[y_hc3 == 1, 0], dfh_val[y_hc3 == 1, 1], s = 100, c = 'green', edgecolor = 'black', label='Cluster 2')
plt.scatter(dfh_val[y_hc3 == 2, 0], dfh_val[y_hc3 == 2, 1], s = 100, c = 'blue', edgecolor = 'black', label='Cluster 3')
plt.title('Hospital beds and GDP per capita (Hierarchical - 3 Clustering Analysis)', fontsize = 18, fontweight = 'bold')
plt.xlabel('Hospital beds', fontsize = 14, fontweight = 'bold')
plt.ylabel('GDP per capita', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize=12,fontweight='bold')
plt.yticks(fontsize=12,fontweight='bold')
plt.legend(loc='best', fontsize = 12)
plt.show()

### 6. Discussion

###  Provide a brief discussion (~ 300 words) on the COVID deaths with respect to Countries economic activity and demographics based on the data analysis results. 

####  France, Russia, Spain, United Kingdom are the worst effected countries due to the pandemic as they reported the highest number of cases and deaths in Europe.  The pandemic had a wide impact on the economies of several countries with respect to the GDP per capita and number of deaths.  There is a negative correlation with respect to the hospital beds and GDP per capita of the countries.  Countries like San Marino, luxembourg, Switzerland, Norway couldn't afford to provide sufficient hospital beds inspite of higher GDP per capita.  The scatter plot between the number of hospital beds and GDP per capita divides the data into three clusters: countries with high GDP per capita have less hospital beds, countries with medium GDP per capita have medium hospital beds and countries with low GDP per capita have more number of hospital beds.   The number of deaths and median are not correlated.  But there is a correlation between the number of deaths and aged 65 to 70 or older.  